In [5]:
import sys
!{sys.executable} -m pip install biopython

     |████████████████████████████████| 2.3 MB 345 kB/s eta 0:00:01


# Biopython - NCBI Blast
We live in an interesting time for biologist. With the advances of sequencing technology the cost to sequence the genomes of a species can be low ( approximately $1000 ). While initiatives have been started for many niches on Earth. The most ambitious being the Earth Biogenome project, seeking to sequence the whole genomes of all life on Earth. A local project, the Darwin Tree of Life, a country specific project to sequence all known species in Ireland and the UK. What I am currently interested in is the 1000 Fungi Genomes project which has a very nice portal.

Some interesting genes from Fungi Genomes have been recently sequenced. However, as sequencing becomes more and more economical a myriad of fungi genomes will become publicly available. Therefore, I am interested in setting up a pipline to detect new species that harbour the orthologs or xenologs.

Orthologs are genes that were shared by the last common ancester and therefore have likely diverged in the species. Xenologs, and this is something unique to the genes we are looking at today, maybe the identical genes moved horizantal between species.

I will be using python3.8 and biopython for this tutorial. To simply install issue the following command with your prefered python installation
```
python -m pip install biopython
```


Blast is an acroynm for The Basic Local Alignment Search Tool. Now, when I say I want to BLAST the gene I am refering to using the NCBI web service to check if there are local alignments in Genbank. The amount of knowledge available through this services is mind blowing and can be found <a href="https://blast.ncbi.nlm.nih.gov/Blast.cgi?PROGRAM=blastn&PAGE_TYPE=BlastSearch&LINK_LOC=blasthome">here</a>

We will also need the nulceotide gene of our interest. Luckliy the NCBI Nucleotide database has a reference to our gene of choice under reference <a href="https://www.ncbi.nlm.nih.gov/nuccore/KY984103.1"> KY984103.1 </a>


First lets download the sequence we want to search for. I already found a sequence of my gene of interest from the nucleotide database. Biopython allows for a a SeqIO object to wrap the Genbank record. This makes handling the data very easy
```
from Bio import Entrez
from Bio import SeqIO
gbAccession = "KY984103.1"
response = Entrez.efetch(db="nucleotide",id=gbAccession,rettype="gb", retmode="text")
record = SeqIO.parse(response, "gb")
```

In [16]:
from Bio import Entrez
from Bio import SeqIO
gbAccession = "KY984103.1"
Entrez.email = "holyflyingfungi@proton.com"
response = Entrez.efetch(db="nucleotide",id=gbAccession,rettype="gb", retmode="text")
seqRecord = SeqIO.read(response, "gb")

The 'record' object here contains the SeqIO object. It can return a generator that contains the seqRecord we want.
```
generator = record.records
seqRecord = next(generator)
seq = seqRecord.seq
print(seqRecord.name + "\n" + seq)
```

In [17]:
seq = seqRecord.seq
print(seqRecord.name + "\n" + seq)

KY984103
ATGCATATCAGGAACCCATACCGCGATGGTGTTGACTACCAAGCACTCGCTGAAGCATTTCCGGCTCTCAAACCACATGTCACAGTAAATTCAGACAATACGACCTCCATCGACTTTGCTGTGCCAGAAGCCCAAAGACTGTATACAGCTGCCCTTCTACACCGGGATTTCGGTCTTACGATCACACTCCCGGAAGACCGTCTTTGTCCGACAGTGCCTAATCGGCTCAACTATGTCCTTTGGGTTGAAGATATCCTTAAAGTCACTTCTGATGCTCTCGGTCTTCCGGATAATCGTCAAGTTAAGGGGATCGATATCGGAACTGGCGCATCAGCGATATATCCCATGCTCGCATGCTCTCGTTTTAAGACATGGTCCATGGTTGCAACAGAGGTAGACCAGAAGTGTATTGACACTGCTCGTCTCAACGTCATTGCCAACAACCTCCAAGAACGTCTCGCAATTATAGCCACCTCCGTCGATGGTCCTATACTTGTCCCCCTCTTGCAGGCGAATTCTGATTTTGAGTACGATTTTACGATGTGTAATCCGCCCTTCTACGATGGGGCATCCGACATGCAGACATCGGATGCTGCGAAGGGGTTTGGATTCGGTGTGAACGCTCCGCATACCGGCACGGTGCTCGAGATGGCCACCGAGGGAGGTGAATCGGCCTTCGTAGCCCAAATGGTCCGCGAAAGTTTGAATCTTCAAACACGATGCAGGTGGTTCACGAGTAATTTGGGGAAATTGAAGTCCTTGTACGAAATTGTGGGGCTGCTGCGAGAACATCAGATAAGTAACTACGCAATCAACGAATACGTCCAAGGAGCCACTCGTCGATATGCGATTGCATGGTCGTTCATCGATGTTCGACTGCCTGATCATTTGTCCCGTCCATCTAACCCCGACCTAAGCTCTCTTTTCTAG


Alright, now we can use that sequence to do a blastn search. A blastn search is a nulceatide input that search for nucleatide matches. The query will take several seconds to complete, but considering the vast amount of searching it is very fast.

We use the blasn algorithm over the metablast algorithm because we are looking for gene relations between species. Discontigous megablast could have worked as well and is faster, but we might not find potential matches.
```
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

blastn_handle = NCBIWWW.qblast(program="blastn", 
                                 database="nt", 
                                 sequence=seq,
                                 megablast=False)
                                 
blastn_results = blastn_handle.read()
with open('results.xml', 'w') as save_file:
    save_file.write(blastn_results)
    
blastn_records = NCBIXML.read(open("results.xml"))
for record in blastn_records
for description in blastn_records.descriptions:
    print(description)
```

The query takes several minutes. Therefore, it's nice to save the output in case we over write the variable. The last part will print out all the descriptions in our records.

In [18]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML


In [19]:
blastn_handle = NCBIWWW.qblast(program="blastn", 
                                 database="nt", 
                                 sequence=seq,
                                 megablast=False)
blastn_results = blastn_handle.read()
with open('results.xml', 'w') as save_file:
    save_file.write(blastn_results)

In [20]:
blastn_records = NCBIXML.read(open("results.xml"))

In [42]:
for description in blastn_records.descriptions:
    print(description)


gi|1234153901|gb|KY984103.1| Psilocybe cyanescens strain FSU 12416 norbaeocystin methyltransferase (psiM) mRNA, complete cds 1860.0  0.0
gi|1234153895|gb|KY984100.1| Psilocybe cubensis strain FSU 12409 norbaeocystin methyltransferase (psiM) mRNA, complete cds 680.0  3.43231e-171
gi|1699065741|gb|MH483014.1| Psilocybe cubensis isolate PC2 norbaeocystin methyltransferase (psiM) gene, partial cds 106.0  2.18138e-15
gi|327304099|ref|XM_003236694.1| Trichophyton rubrum CBS 118892 hypothetical protein (TERG_03786) mRNA, complete cds  67.0  0.00015707
gi|695566229|ref|XM_009551449.1| Heterobasidion irregulare TC 32-1 hypothetical protein partial mRNA  66.0  0.00015707
gi|1768673944|emb|LR732079.1| Armillaria ostoyae strain C18/9 genome assembly, chromosome: LG5  61.0  0.0066788
gi|528078200|ref|NM_001281658.1| Mesocricetus auratus aryl hydrocarbon receptor (Ahr), mRNA >gi|9438130|gb|AF275721.1|AF275721 Mesocricetus auratus AH receptor mRNA, complete cds  55.0  0.28399
gi|9188538|dbj|AB033505.

The last two number are the score and the e value, respectively. Now, what you should know about the score is it is an aligment score and the more correct the alignment the higher the score. As you can see in the first entry has the highest score from any entry. In fact, if you look at the accession KY984103.1 it is infact the seq we downloaded orginally. For those famliar with statistics the e value is a significant score. It is defined as: 
<em>
 the number of alignments expected by chance with the calculated score or better. The expect value is the default sorting metric; for significant alignments the E value should be very close to zero.
 </em>
 That is, the closer the score is to zero the more likely these sequences are not aligning by chance. For e scores that are not close to one or where the alignment score is very low ( the cut off by default seems to be set to fifty ) are noise. Chance aligments. Trichophyton rubrum is the Ascomycota responsible for athletes foot. Perhaps a very distant ortholog exists here, I'm not sure. I do not understand the origin of this gene well enough, but I would account this to noise. It would be easy to do a basic filter locally. Let's do an aligment score of atleast 100 and an a e score of the .00005. We can pull out the top aligment and display it as well.
 
 ```
  for description, alignment in zip(blastn_records.descriptions, blastn_records.alignments):
    if description.e < .00005 and description.score > 100:
            print(description)
            print(alignment.hsps[0])
 ```

In [44]:
for description, alignment in zip(blastn_records.descriptions, blastn_records.alignments):
    if description.e < .00005 and description.score > 100:
            print(description)
            print(alignment.hsps[0])

gi|1234153901|gb|KY984103.1| Psilocybe cyanescens strain FSU 12416 norbaeocystin methyltransferase (psiM) mRNA, complete cds 1860.0  0.0
Score 1860 (1678 bits), expectation 0.0e+00, alignment length 930
Query:       1 ATGCATATCAGGAACCCATACCGCGATGGTGTTGACTACCAAGCA...TAG 930
               |||||||||||||||||||||||||||||||||||||||||||||...|||
Sbjct:       1 ATGCATATCAGGAACCCATACCGCGATGGTGTTGACTACCAAGCA...TAG 930
gi|1234153895|gb|KY984100.1| Psilocybe cubensis strain FSU 12409 norbaeocystin methyltransferase (psiM) mRNA, complete cds 680.0  3.43231e-171
Score 680 (614 bits), expectation 3.4e-171, alignment length 930
Query:       1 ATGCATATCAGGAACCCATACCGCGATGGTGTTGACTACCAAGCA...TAG 930
               ||||||||||| || || |||||        ||||||| ||||||...|||
Sbjct:       1 ATGCATATCAGAAATCCTTACCGTACACCAATTGACTATCAAGCA...TAG 930
gi|1699065741|gb|MH483014.1| Psilocybe cubensis isolate PC2 norbaeocystin methyltransferase (psiM) gene, partial cds 106.0  2.18138e-15
Score 106 (96 bits), expectation 2.

We have some basic filtered results. Now lets simplify the code and leverage the features of the library. First, we can use the accession directly in the Blast algorith. We do not need to download it separately. Second, we can pass filter options directly to the Blast agorithm. This is nice if you have thousands of hits. However, there is no fast and hard cut offs. One should discovery these first and then apply them.

In [49]:
accession = "KY984103.1"
expectCutOff = .00005
blastn_results = NCBIWWW.qblast(program="blastn", 
                                 database="nt", 
                                 sequence= accession,
                                 expect = expectCutOff,
                                 megablast=False)
blastn_records = NCBIXML.read(blastn_results)

In [51]:
for description, alignment in zip(blastn_records.descriptions, blastn_records.alignments):
            print(description)
            print(alignment.hsps[0])

gi|1234153901|gb|KY984103.1| Psilocybe cyanescens strain FSU 12416 norbaeocystin methyltransferase (psiM) mRNA, complete cds 1860.0  0.0
Score 1860 (1678 bits), expectation 0.0e+00, alignment length 930
Query:       1 ATGCATATCAGGAACCCATACCGCGATGGTGTTGACTACCAAGCA...TAG 930
               |||||||||||||||||||||||||||||||||||||||||||||...|||
Sbjct:       1 ATGCATATCAGGAACCCATACCGCGATGGTGTTGACTACCAAGCA...TAG 930
gi|1234153895|gb|KY984100.1| Psilocybe cubensis strain FSU 12409 norbaeocystin methyltransferase (psiM) mRNA, complete cds 680.0  3.43231e-171
Score 680 (614 bits), expectation 3.4e-171, alignment length 930
Query:       1 ATGCATATCAGGAACCCATACCGCGATGGTGTTGACTACCAAGCA...TAG 930
               ||||||||||| || || |||||        ||||||| ||||||...|||
Sbjct:       1 ATGCATATCAGAAATCCTTACCGTACACCAATTGACTATCAAGCA...TAG 930
gi|1699065741|gb|MH483014.1| Psilocybe cubensis isolate PC2 norbaeocystin methyltransferase (psiM) gene, partial cds 106.0  2.18138e-15
Score 106 (96 bits), expectation 2.